## Basic Workflow

This is a demonstration implementation of multi-year reconciliation. Reconciliation is performed by specifying a two-element `reconciliation range`. The parameters are changed from the first year in the reconciliation range, and the reconciliation objective function is calculated over all years in the range. If both entries in the reconciliation range are the same, it is functionally equivalent to old-style reconciliation with `reconcile_for_year` set to that same year. 

In [1]:
# IMPORTS
%load_ext autoreload
%autoreload 2
%matplotlib inline
import sys
sys.path.append('..')
from optima_tb.project import Project
from optima_tb.utils import odict
from optima_tb.plotting import plotResult
from optima_tb.defaults import defaultOptimOptions
import optima_tb.plotting as oplt
import optima_tb.plotting2 as oplt2
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, HTML
from optima_tb.plotting2 import plotSeries, PlotData



Optima TB: a TB optimization and analysis tool
Copyright (C) 2017 by the Optima Consortium



In [2]:
import logging
logger = logging.getLogger()
logger.setLevel('WARN')

In [3]:
cascade = '../../tb-ucl-analyses/belarus/Cascade Spreadsheets/cascade-belarus.xlsx'
databook = '../../tb-ucl-analyses/belarus/Databook Spreadsheets/databook-belarus.xlsx'
proj= Project(name = 'Belarus', cascade_path = cascade, validation_level = 'avert', plotting_level = 'dev')
proj.loadSpreadsheet(databook_path=databook)
parset = proj.makeParset(name='default')
progset = proj.makeProgset(name='default-progset')
parset_results = proj.runSim(parset_name='default', plot=False)
options = defaultOptimOptions(settings=proj.settings, progset=proj.progsets[0])
default_results = proj.runSim(parset_name='default', progset_name='default-progset', options=options)
parset_results.name = 'Parset'
default_results.name = 'Unreconciled'

In [4]:
from optima_tb.reconciliation4 import reconcile
parset_name='default'
progset_name = 'default-progset'
unitcost_sigma=0.05
attribute_sigma=0.2
budget_sigma = 0.0
impact_pars=None

sigma_dict=None, unitcost_sigma=0.05, attribute_sigma=0.20

SyntaxError: cannot assign to None (<ipython-input-4-edab17edeffc>, line 9)

Now, reconciliation mainly depends on the `reconciliation_range`. Some options would be
- Setting both to be the same, which is the same behaviour as before
- Setting the lower bound to be prior to the program start year
- Setting the upper bound to be after the program start year
- Setting the range to span the program start year

We will now evaluate each of these options in turn

In [ ]:
def test_reconciliation(reconciliation_range):
    reconciled_progset,outcome = reconcile(proj,parset_name,progset_name,reconciliation_range,impact_pars=None,unitcost_sigma=unitcost_sigma, attribute_sigma=attribute_sigma, budget_sigma = budget_sigma,max_time=20,verbose=0)
    reconciled_results = proj.runSim(parset_name=parset_name, progset=reconciled_progset, options=options, plot=False)
    reconciled_results.name = 'Reconciled'
    d = PlotData([parset_results,default_results,reconciled_results],outputs=['pxtreat'], pops=['15-64'])
    d.set_colors('b',results=['Parset'])
    d.set_colors('r',results='Unreconciled')
    d.set_colors('g',results='Reconciled')
    figs = plotSeries(d,axis='results');
    ax = figs[0].axes[0]
    ax.axvspan(reconciliation_range[0],reconciliation_range[1], alpha=0.1, facecolor='red',edgecolor='none',linewidth=0.0)
    ax.axvline(x=options['progs_start'],color='k',linestyle='--', alpha=0.5,linewidth=0.2); # Highlight a particular time on the plot
    plt.show()

### Running standard reconciliation multiple times

The budget after the program start year (black line) is unchanged, so the choice of reconciliation could have a significant effect on predictions going forward

In [ ]:
test_reconciliation([2015,2015])

In [ ]:
test_reconciliation([2015,2015])

In [ ]:
test_reconciliation([2015,2015])

### Reconciling prior to start year

The idea is that we start reconciling a few years in advance, comparing the known parameter data to the known program data

In [ ]:
test_reconciliation([2012,2015])

In [ ]:
test_reconciliation([2012,2015])

In [ ]:
test_reconciliation([2012,2015])

### Reconciling after start year

The idea is that the parset projections reflect business as usual, including the effect of any programs that were actually active in the last parset year, so if the program continues being funded in the same way as in the reconciliation year/program start year, then it should approximately match the parset

In [ ]:
test_reconciliation([2015,2018])

In [ ]:
test_reconciliation([2015,2018])

In [ ]:
test_reconciliation([2015,2018])

### Reconciling spanning start year

In [ ]:
test_reconciliation([2012,2018])

In [ ]:
test_reconciliation([2012,2018])

In [ ]:
test_reconciliation([2012,2018])

### Reconciliation over a longer time period

In [ ]:
test_reconciliation([2015,2023])

In [ ]:
test_reconciliation([2015,2023])

In [ ]:
test_reconciliation([2015,2023])